Pipeline
- deseason features (from statsmodels.tsa and notebook 3.3.1)
- [lasso linear](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html)
  - good at filtering out features when there are many of them
- either lasso directly on target, or just use lasso for feature reduction and use another model like RF or OLS
- append t-1 and t-2
- use polynomial interactions of degree=3 but without t-1/t-2 or degree=2 with them (with/without for memory purposes)
- use log10 transformation on target for training

TODO
- correlation matrix like in the [benchmark](https://github.com/drivendata/benchmarks/blob/master/dengue-benchmark-statsmodels.ipynb)
- check [GaussianProcessRegressor](http://scikit-learn.org/stable/modules/generated/sklearn.gaussian_process.GaussianProcessRegressor.html)
- check [related projects](http://scikit-learn.org/stable/related_projects.html) that handle sequences more properly (check `n_back` variable)
  - [seqlearn](https://github.com/larsmans/seqlearn)
  - [hmmlearn](http://hmmlearn.readthedocs.io/en/latest/auto_examples/plot_hmm_stock_analysis.html#sphx-glr-auto-examples-plot-hmm-stock-analysis-py)

Result
- the submission has one epidemic for SJ and none for IQ
- having t-1 and t-2 appended doesn't help identify more epidemics
- having polynomial degree = 3 also
- keeping trend and residual from deseasoning is good
- RF gives a smoother "bump", albeit smaller amplitude
- t-0 .. t-10 without polynomial and RF yields larger amplitude
- lasso cross-validation doesn't converge within 10k iterations

Submissions
- 5.1A .. lasso + RF .. score 30
- 5.1B .. lasso + linear .. score 29
- 5.1C .. lasso + linear with t-0..t-50 .. score 38
- 5.1D .. lasso + RF with t-0..t-50 .. did not submit since no epidemics detected

In [ ]:
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import time

# http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.linear_model import Lasso, LinearRegression, LassoCV
from sklearn.preprocessing import MinMaxScaler, FunctionTransformer, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel

#import statsmodels.api as sm
#import statsmodels.formula.api as smf

In [ ]:
from src.features.build_features import load_raw

df_all = load_raw()

# replace with 0.2 output
# df_all['labels_train'] = pd.read_pickle('data/processed/is_epidemic.pkl')

[(x, df_all[x].shape) for x in df_all.keys()]

In [ ]:
df_all['features_train'].head(n=2)

In [ ]:
df_all['labels_train'].head(n=2)

## gather all data into a single dataframe

With this, deseasoning loses 52 points only once, instead of 3 times (train, test, submit)

In [ ]:
df_all['features_train']['submit'] = False
df_all['features_test' ]['submit'] = True

df_feat_1 = pd.concat([df_all['features_train'], df_all['features_test'], ], axis=0)
df_targ   = pd.concat([df_all['labels_train'], df_all['submission'], ], axis=0)
# df_one = pd.concat([df_feat, df_targ[['total_cases']]], axis=1)

df_meta = df_feat_1[['submit']]
del df_feat_1['submit']
del df_feat_1['year']
# del df_feat_1['weekofyear']

df_feat_1.shape, df_targ.shape, df_meta.shape

In [ ]:
# df_one[df_one['total_cases']!=0].groupby('city').head(n=2)
df_feat_1.groupby('city').head(n=2)

In [ ]:
df_targ.groupby('city').head(n=2)

In [ ]:
df_meta.groupby('city').head(n=2)

In [ ]:
df_meta.groupby('city').tail(n=2)

## fillna

In [ ]:
df_feat_1 = df_feat_1.groupby('city').apply(lambda group: group.fillna(method='ffill'))
assert ~(pd.isnull(df_feat_1).any().any())
print(df_feat_1.shape)

## define custom model for deseasoning

In [ ]:
from sklearn.base import BaseEstimator
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.utils import check_array


class DeSeason(BaseEstimator):
    def __init__(self, freq):
        self.freq = freq
        
    def fit(self):
        pass
    
    def transform(self, df_in):
        return self.fit_transform(df_in)

    def fit_transform(self, X, y=None):
        X = check_array(X)
        df_interim = []
        for jjj in range(X.shape[1]):
            res0 = X[:,jjj]
            res1 = res0 - res0.mean(axis=0)
            res2 = seasonal_decompose(res1, freq=self.freq, two_sided=False)
            res2 = pd.DataFrame({
                #'original': res0,
                'trend': res2.trend, 
                # FIXME # 'seasonal': res2.seasonal, 
                'resid': res2.resid
                # 'chosen': res2.trend + res2.resid
            })

            # FIXME # res2['original'] = res0
            res2 = res2.rename(columns={
                #'original': "%s_original"%jjj,
                'trend': "%s_trend"%jjj,
                #'seasonal': "%s_seasonal"%jjj,
                'resid': "%s_resid"%jjj,
                #'chosen': "%s_deseason"%jjj,
            })
            df_interim.append(res2)
            
        # aggregate
        df_interim = pd.concat(df_interim, axis=1)
            
        # MOVE SEQUENCING till after polynomial interactions
        # break into t and t-1 and t-2
        # col_new = lambda k: {x: "%s_t%s"%(x,k) for x in df_interim.columns}
        # df_tm = []
        # for m in range(20):
        #    df_tm.append(df_interim.shift(m).rename(columns=col_new(m)))
        #    
        #df_interim = pd.concat(df_tm, axis=1)
        
        # fillna
        df_interim = df_interim.fillna(value=0)

        return df_interim
    
# test
mdl = DeSeason(freq=2)
df_in = np.array([
    [1.0,2.0,3.0],[4.0,5.0,6.0],
    [1.1,2.0,3.0],[4.1,5.0,6.0],
    [1.2,2.0,3.0],[4.2,5.0,6.0],
    [1.3,2.0,3.0],[4.3,5.0,6.0],
])
df_out = mdl.fit_transform(df_in)
df_out

## preprocess
deseason + min/max + polynomial interactions

In [ ]:
def create_preprocess():
    m0 = DeSeason(freq=52)
    
    # http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html#sklearn.preprocessing.MinMaxScaler
    m1 = MinMaxScaler(feature_range=(0, 10))
    
    # FIXME disabling because lasso for iq at alpha=.1 or .5 was too slow
    m2 = PolynomialFeatures(degree=2) # FIXME degree=3 runs out of memory with t-1 and t-2
    
    model = Pipeline([
        ('deseason', m0),
        ('scaler', m1),
        ('poly', m2),
    ])
    return model


mod0 = {}
df_feat_2 = []
for city in ['sj','iq']:
    mod0[city] = create_preprocess()
    df_temp = mod0[city].fit_transform(X = df_feat_1.loc[city], y = None)
    df_temp = pd.DataFrame(
        df_temp, 
        columns=mod0[city].named_steps['poly'].get_feature_names(), 
        index=df_feat_1.loc[city].index
    )
    df_temp['city'] = city
    df_feat_2.append(df_temp.reset_index().set_index(['city','week_start_date']))
    
df_feat_2 = pd.concat(df_feat_2, axis=0)
df_feat_2 = df_feat_2.loc[df_meta.index] # re-index as original

df_feat_2.shape

## build sequences
t-0, t-1, t-2, ...

In [ ]:
n_back = 50

col_new = lambda k: {x: "%s_t%s"%(x,k) for x in df_feat_2.columns}
df_tm = []
for m in range(n_back):
    print('sequence %s'%m)
    df_tm.append(df_feat_2.shift(m).rename(columns=col_new(m)))

df_feat_2 = pd.concat(df_tm, axis=1)

# fillna
df_feat_2 = df_feat_2.fillna(value=0)


## preview

In [ ]:
df_feat_2.groupby('city').head(n=2)

## choose features

In [ ]:
# features selected from
# https://github.com/drivendata/benchmarks/blob/master/dengue-benchmark-statsmodels.ipynb
#selected_features = ['reanalysis_specific_humidity_g_per_kg', 
#                 'reanalysis_dew_point_temp_k', 
#                 'station_avg_temp_c', 
#                 'station_min_temp_c']

# all features
# selected_features = df_all['features_train'].columns
selected_features = df_feat_2.columns

# without year and weekofyear
# selected_features = np.array(list(set(df_all['features_train'].columns) - set(['year', 'weekofyear'])))

# check no missing
# assert len(set(selected_features) - set(df_all['features_train'].columns))==0

#################################

# all original/trend/seasonal features
# selected_features = df_train.columns

# only trend + weekofyear
# import numpy as np
# selected_features = np.array([x for x in df_train.columns if x.endswith('_trend')])# or x=='weekofyear'])

#################

selected_features

## plot

In [ ]:
df_train = df_feat_2

## train/test split

In [ ]:
# split per city
x_train = (#df_all['features_train']
           df_train.loc[~df_meta['submit']]
          .groupby(level='city', as_index=False)
          .apply(lambda group: group.head(n=group.shape[0]*7//8))
          .reset_index(level=0, drop=True)
          # FIXME for no split, comment "apply" and "reset above", and uncomment "apply" below
          #.apply(lambda group: group)
          [selected_features]
          )
x_test = (#df_all['features_train']
          df_train.loc[~df_meta['submit']]
          .groupby(level='city', as_index=False)
          .apply(lambda group: group.tail(n=group.shape[0]*1//8))
          .reset_index(level=0, drop=True)
          [selected_features]
         )
y_train = ( #df_all['labels_train']
            #df_all['labels_train'].loc[df_train.index]
            df_targ.loc[~df_meta['submit']]
          .groupby('city', as_index=False)
           .apply(lambda group: group.head(n=group.shape[0]*7//8))
          .reset_index(level=0, drop=True)
          # FIXME for no split, comment "apply" and "reset above", and uncomment "apply" below
          # .apply(lambda group: group)
          ['total_cases']
          # ['is_epidemic'].astype('int')
         )
y_test = ( #df_all['labels_train']
            #df_all['labels_train'].loc[df_train.index]
            df_targ.loc[~df_meta['submit']]
          .groupby('city', as_index=False)
          .apply(lambda group: group.tail(n=group.shape[0]*1//8))
          .reset_index(level=0, drop=True)
          ['total_cases']
          # ['is_epidemic'].astype('int')
         )

y_train = 100*np.log10(y_train+1)
y_test = 100*np.log10(y_test+1)

x_train.shape, x_test.shape, y_train.shape, y_test.shape

In [ ]:
x_train.groupby('city').head(n=2)

In [ ]:
x_test.groupby('city').head(n=2)

In [ ]:
set(y_train.reset_index()['city'])

In [ ]:
y_train.groupby('city').describe()#tail(n=15)

In [ ]:
y_test.groupby('city').tail(n=2)

In [ ]:
df_meta.groupby('city').tail(n=2)

## fit


In [ ]:
def create_model(alpha):
    # return RandomForestRegressor(n_estimators=100, min_samples_split=5, min_samples_leaf=3)
    # return RandomForestClassifier(n_estimators=100, min_samples_split=5, min_samples_leaf=3)
    # return Lasso(alpha=1., normalize=True)
    
    # http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html#sklearn.linear_model.Lasso
    # max_iter default is 1k, but with t-1/t-2 and polynomials degree = 2, didnt converge
    m31 = Lasso(alpha=alpha, normalize=False, positive=True, max_iter=10000)
    
    # use cross-validation: causes lasso not to converge within 10k iterations
    # m31 = LassoCV(normalize=False, positive=False, max_iter=10000)
    
    # http://scikit-learn.org/stable/modules/feature_selection.html#l1-based-feature-selection
    m32 = SelectFromModel(m31, prefit=False)
    # m33 = RandomForestRegressor(n_estimators=100, min_samples_split=5, min_samples_leaf=3)
    m33 = RandomForestRegressor(n_estimators=500, min_samples_split=15, min_samples_leaf=13)
    
    # m33 = LinearRegression(fit_intercept=False) # already appended intercept with polynomial
    
    model = Pipeline([
        #('regressor', m31),
        ('reducer', m32),
        ('regressor', m33),
    ])
    # model.set_params(anova__k=10, svc__C=.1).fit(X, y)
    return model

lasso_settings = [
    ('sj',1.),
    # FIXME had .1 for iq when worked with Jessica
    # Too slow with t-1..t-10
    # Bumped up to speed up convergence
    ('iq',.1),
]

mod1 = {}
for city, alpha in lasso_settings:
    # if city=='sj': continue
    print(time.ctime(), city, 'fit start')
    mod1[city] = create_model(alpha=alpha)
    mod1[city].fit(X = x_train.loc[city], y = y_train.loc[city])
    print(time.ctime(), city, 'fit end')
    
mod1

## check feature importances

In [ ]:
selected_features[[1,10]]

## predict on train to visualize

In [ ]:
# cast to int since we know the label is integer
predictions = (y_train.copy()*0).astype('int')

for city in ['sj','iq']:
    predictions.loc[city] = mod1[city].predict(x_train.loc[city])
    #predictions = 10**predictions.astype('int')

for city in ['sj', 'iq']:
    plt.plot(y_train.loc[city], label='actual')
    plt.plot(predictions.loc[city], label='predicted')
    plt.title(city)
    plt.legend()
    plt.show()

## predict on test set

In [ ]:
# cast to int since we know the label is integer
predictions = (y_test.copy()*0).astype('int')

for city in ['sj','iq']:
    predictions.loc[city] = mod1[city].predict(x_test.loc[city])#.astype(int)
    
    # using sj model for iq
    # predictions.loc[city] = mod1['sj'].predict(x_test.loc[city])#.astype(int)

#predictions = (10**predictions).astype('int')
# predictions.loc['sj'].head()

for city in ['sj', 'iq']:
    plt.plot(y_test.loc[city], label='actual')
    plt.plot(predictions.loc[city], label='predicted')
    plt.title(city)
    plt.legend()
    plt.show()

In [ ]:
'sj', mod1['sj'].score(x_test.loc['sj'], y_test.loc['sj']), 'iq', mod1['iq'].score(x_test.loc['iq'], y_test.loc['iq'])

## re-fit on complete dataset

In [ ]:
df_feat_2.shape, df_meta['submit'].shape

In [ ]:
df_feat_2.loc['sj'].head()

In [ ]:
df_meta.loc['sj'].head()

In [ ]:
x_retrain = df_feat_2[selected_features][~df_meta['submit']]
y_retrain = df_targ['total_cases'][~df_meta['submit']]
y_retrain = 100*np.log10(y_retrain+1)

mod2 = {}
for city, alpha in lasso_settings:
    print(time.ctime(), city, 'fit start')
    mod2[city] = create_model(alpha=alpha)
    mod2[city].fit(X = x_retrain.loc[city], y = y_retrain.loc[city])
    print(time.ctime(), city, 'fit start')
    
mod2

In [ ]:
df_test = df_feat_2[df_meta['submit']]

df_test.shape, df_targ['total_cases'].shape

In [ ]:
'weekofyear' in df_test.columns, 'weekofyear' in df_train.columns

## set in submission

In [ ]:
df_feat_2.shape, df_meta.shape, df_train.shape

In [ ]:
# cast to int since we know the label is integer
predictions = (df_all['submission'][['total_cases']]*0).astype('int')

p1 = mod2['sj'].predict(df_test.loc['sj', selected_features])
p1 = pd.DataFrame({'pred': p1, 'city': 'sj', 'week_start_date': df_test.loc['sj'].index})
p2 = mod2['iq'].predict(df_test.loc['iq', selected_features])
p2 = pd.DataFrame({'pred': p2, 'city': 'iq', 'week_start_date': df_test.loc['iq'].index})

p3 = pd.concat([p1,p2], axis=0)
p3 = p3.set_index(['city', 'week_start_date'])

predictions = predictions.merge(p3, left_index=True, right_index=True, how='left').fillna(value=0)
# predictions['pred'] = 10**predictions['pred'].astype('int')
predictions['total_cases'] = predictions['pred']
del predictions['pred']

# postprocess to match with original format
predictions['total_cases'] = ((10**((predictions['total_cases']/100).clip(upper=200000)))-1).astype('int')

predictions.head(n=60).tail(n=5)

In [ ]:
submit = df_all['submission'].copy()
# TODO Will this match indeces properly?
# submit['total_cases'] = predictions

del submit['total_cases']

submit = submit.merge(
    predictions,
    left_index=True,
    right_index=True,
    how='left'
)
submit['total_cases'] = submit['total_cases'].fillna(value=0)

In [ ]:
submit.shape

In [ ]:
submit.groupby('city').head(n=2)

## plot

In [ ]:
for city in ['sj','iq']:
    submit.loc[city, 'total_cases'].plot(figsize=(20,3), label=city)
        
plt.legend()
plt.show()

## generate submission file

In [ ]:
from src.features.build_features import make_submission

In [ ]:
make_submission(submit.reset_index())